In [1]:
import pandas as pd

In [111]:
knvk_mdg = pd.read_excel('../sync/c-after-sync/mdg/knvk.xlsx', dtype=str)
knvk_ecc = pd.read_excel('../sync/c-after-sync/ecc/knvk.xlsx', dtype=str)

In [112]:
print(len(knvk_ecc.columns))
print(len(knvk_mdg.columns))

74
74


In [113]:
cols_mdg = set(knvk_mdg.columns.str.lower())
cols_ecc = set(knvk_ecc.columns.str.lower())

In [114]:
only_in_mdg = sorted(cols_mdg - cols_ecc)
only_in_ecc = sorted(cols_ecc - cols_mdg)

In [115]:
df_only_mdg = pd.DataFrame({'In mdg not in ECC': only_in_mdg})
df_only_ecc = pd.DataFrame({'In ECC not in mdg': only_in_ecc})

In [116]:
print(df_only_ecc)
print(df_only_mdg)

  In ECC not in mdg
0          supplier
  In mdg not in ECC
0            vendor


In [117]:
with pd.ExcelWriter('../sync/c-after-sync/column_mismatch_knvk.xlsx', engine='openpyxl') as writer:
    df_only_mdg.to_excel(writer, sheet_name='In mdg', index=False)
    df_only_ecc.to_excel(writer, sheet_name='In ECC', index=False)

In [ ]:
n_cols_ecc = list(set(knvk_ecc.columns) - (set(knvk_ecc.columns) - set(knvk_mdg.columns)))
n_cols_mdg = list(set(knvk_mdg.columns) - (set(knvk_mdg.columns) - set(knvk_ecc.columns)))

In [73]:
n_cols_ecc.append('Supplier')
n_cols_mdg.append('Vendor')

In [ ]:
knvk_ecc = knvk_ecc[n_cols_ecc]
knvk_mdg = knvk_mdg[n_cols_mdg]

In [118]:
print(knvk_ecc.columns)
print(knvk_mdg.columns)

Index(['Contact Person', 'Customer', 'First name', 'Name 1', 'City',
       'Business address', 'Home address', 'Customer department', 'Department',
       'Higher-level person', 'Telephone 1', 'Form of address', 'Function',
       'Authority', 'VIP', 'Gender', 'Language', 'Date of birth',
       'Representative no.', 'Call frequency', 'Buying habits',
       'Advertising material', 'Notes about contact person', 'Attribute 1',
       'Attribute 2', 'Attribute 3', 'Attribute 4', 'Attribute 5', 'Monday',
       'Monday.1', 'Monday.2', 'Monday.3', 'Tuesday', 'Tuesday.1', 'Tuesday.2',
       'Tuesday.3', 'Wednesday', 'Wednesday.1', 'Wednesday.2', 'Wednesday.3',
       'Thursday', 'Thursday.1', 'Thursday.2', 'Thursday.3', 'Friday',
       'Friday.1', 'Friday.2', 'Friday.3', 'Saturday', 'Saturday.1',
       'Saturday.2', 'Saturday.3', 'Sunday', 'Sunday.1', 'Sunday.2',
       'Sunday.3', 'Attribute 6', 'Attribute 7', 'Attribute 8', 'Attribute 9',
       'Attribute 10', 'Search term', 'Marital

In [119]:
knvk_ecc['key'] = knvk_ecc.apply(
    lambda row: 
        str(row['Customer'])
        + str(row['Contact Person']),
        # + str(row['Distribution Channel'])
        # + str(row['Division']),
        # + str(row['Sequence Number']), 
        axis=1
)

In [120]:
knvk_mdg['key'] = knvk_mdg.apply(
    lambda row: 
        str(row['Customer'])
        + str(row['Contact Person']),
        # + str(row['Distribution Channel'])
        # + str(row['Division']),
        # + str(row['Sequence Number']), 
        axis=1
)

In [121]:
print(len(knvk_ecc))
print(len(knvk_mdg))

2417
2403


In [122]:
knvk_ecc.set_index('key', inplace=True)
knvk_mdg.set_index('key', inplace=True)

In [123]:
a_knvk_mdg, a_knvk_ecc = knvk_mdg.align(knvk_ecc, join='outer')
# a_knvk_mdg, a_knvk_ecc = knvk_mdg, knvk_ecc


In [124]:
print(len(a_knvk_ecc))
print(len(a_knvk_mdg))

2417
2417


In [128]:
diffs = a_knvk_mdg.compare(
    a_knvk_ecc,
    keep_equal=True,      # show all cells
    # keep_equal=False,      # only show changed cells
    keep_shape=False,       # retain full shape if you want blanks where no change
    result_names=('MDG','ECC')
)

In [129]:
diffs.columns = [f"{col[0]} ({col[1]})" for col in diffs.columns]

In [130]:
diffs.to_excel('../sync/c-after-sync/knvk_allData_changes.xlsx')
# diffs.to_excel('../sync/c-after-sync/knvk_changes.xlsx')

In [73]:
# Ignore

In [ ]:
f = pd.read_excel('../knvkData_changes.xlsx', sheet_name='Sheet3')

In [148]:
f.head()

,Name,email
0,3M Electro & Communication,pbhutani@mmm.com
1,3M Electro & Communication,srjanak@mmm.com
2,A P ELECTRICALS AND ENGINEERS,SVLNRAO@APEEONLINE.COM
3,A P ELECTRICALS AND ENGINEERS,VIJAY@APEEONLINE.COM
4,AAALAC INTERNATIONAL,msprouse@aaalac.org


In [153]:
def func(df):
    n = df['Name'].split()
    
    for i in n:
        if i.lower() in df['email'].lower():
            return True
    return False

In [154]:
f['new'] = f.apply(lambda row: func(row), axis=1)

In [155]:
f

,Name,email,new
0,3M Electro & Communication,pbhutani@mmm.com,False
1,3M Electro & Communication,srjanak@mmm.com,False
2,A P ELECTRICALS AND ENGINEERS,SVLNRAO@APEEONLINE.COM,True
3,A P ELECTRICALS AND ENGINEERS,VIJAY@APEEONLINE.COM,True
4,AAALAC INTERNATIONAL,msprouse@aaalac.org,True
...,...,...,...
2828,ZELLE BIOTECHNOLOGY PVT LTD,accounts@zellebiotech.com,True
2829,ZELLE BIOTECHNOLOGY PVT LTD,orders@zellebiotech.com,True
2830,ZELLE BIOTECHNOLOGY PVT LTD,service@zellebiotech.com,True
2831,ZoomInfo Technologies LLC,ar@zoominfo.com,True


In [156]:
f.to_csv('email.csv')

In [ ]:
knvkcc.columns

Index(['Address Number', 'from', 'Address Version', 'To', 'Title', 'Name',
       'Name 2', 'Name 3', 'Name 4', 'Converted name (with form of address)',
       'c/o name', 'City', 'District', 'City Code', 'District.1',
       'City (Diff. from Postal City)', 'City Code.1', 'Test stat./City file',
       'Reg. Struct. Grp.', 'Postal Code', 'PO Box Postal Code',
       'Company Postal Code', 'Postl Code Extension', 'Postl Code Extension.1',
       'Postl Code Extension.2', 'PO Box', 'Undeliverable', 'PO Box w/o No.',
       'PO Box City', 'City Code.2', 'PO Box Region', 'PO box country',
       'Delivery District', 'Transportation zone', 'Street', 'Undeliverable.1',
       'Street Code', 'Street Abbreviation', 'House Number', 'Supplement',
       'House Number Range', 'Street 2', 'Street 3', 'Street 4', 'Street 5',
       'Building Code', 'Floor', 'Room Number', 'Country Key', 'Language Key',
       'Region', 'Address group', 'Flag: There are more address group assig',
       'personal a

In [ ]:
knvk.apply(lambda row: '&' in str(row['Street']), axis=1).value_counts()

False    2944
True       78
Name: count, dtype: int64